In [ ]:
!pip install langchain_groq langchain_core langchain_community

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq (
    temperature=0,
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile"
)
result = llm.invoke("what is python")
print(result.content)

**Python Overview**

Python is a high-level, interpreted programming language that is widely used for various purposes such as:

* **Web Development**: Creating web applications and web services using popular frameworks like Django and Flask.
* **Data Analysis**: Analyzing and visualizing data using libraries like Pandas, NumPy, and Matplotlib.
* **Artificial Intelligence**: Building AI and machine learning models using libraries like TensorFlow and Keras.
* **Automation**: Automating tasks and processes using scripts and tools like PyAutoGUI and Robot Framework.
* **Scientific Computing**: Performing scientific simulations and computations using libraries like SciPy and NumPy.

**Key Features of Python**
-------------------------

* **Easy to Learn**: Python has a simple syntax and is relatively easy to learn, making it a great language for beginners.
* **High-Level Language**: Python abstracts away many low-level details, allowing developers to focus on the logic of their code.
* **I

In [ ]:
!pip install pypdf

In [ ]:
!pip install chromadb

In [ ]:
!pip install sentence_transformers

In [ ]:

!pip install -q langchain chromadb ipywidgets
!pip install -q langchain-core langchain-community langchainhub
!pip install -q pypdf


import requests
from langchain_core.language_models import BaseLanguageModel
from langchain_core.messages import HumanMessage
from typing import List

class GroqLLM(BaseLanguageModel):
    def __init__(self, model: str, groq_api_key: str, temperature: float = 0.7):
        self.model = model
        self.api_key = groq_api_key
        self.temperature = temperature

    def _call(self, messages: List[HumanMessage], **kwargs) -> str:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        data = {
            "model": self.model,
            "messages": [{"role": "user", "content": messages[0].content}],
            "temperature": self.temperature
        }
        response = requests.post(
            "https://api.groq.com/openai/v1/chat/completions",
            headers=headers,
            json=data
        )
        if response.status_code != 200:
            raise ValueError(f"Groq API Error: {response.text}")
        return response.json()["choices"][0]["message"]["content"]

    @property
    def _llm_type(self) -> str:
        return "custom_groq_llm"


groq_api_key = "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"
llm = GroqLLM(model="llama-3.3-70b-versatile", groq_api_key="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")


from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


loader = PyPDFLoader("mental_health.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


embedding = OpenAIEmbeddings(openai_api_key="your_openai_api_key_here")
vectordb = Chroma.from_documents(splits, embedding, persist_directory="./chroma_store")
retriever = vectordb.as_retriever(search_kwargs={"k": 3})


from langchain.chains import RetrievalQA
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""
Answer the question based on the provided context.

Context: {context}
Question: {question}
Answer:
""")

stuff_chain = StuffDocumentsChain(
    llm_chain=LLMChain(llm=llm, prompt=prompt),
    document_variable_name="context"
)

qa_chain = RetrievalQA(combine_documents_chain=stuff_chain, retriever=retriever)


import ipywidgets as widgets
from IPython.display import display

input_box = widgets.Text(placeholder='Ask me something...', description='You:', layout=widgets.Layout(width='100%'))
output_area = widgets.Output()

def handle_input(change):
    query = change['new']
    if not query.strip():
        return
    with output_area:
        print(f"\nHuman: {query}")
        try:
            response = qa_chain.run(query)
            print(f"Bot: {response}")
        except Exception as e:
            print(f"Error: {e}")
    input_box.value = ''

input_box.observe(handle_input, names='value')

print("Initializing Mental Health Chatbot...")
display(input_box, output_area)


TypeError: Can't instantiate abstract class GroqLLM with abstract methods agenerate_prompt, apredict, apredict_messages, generate_prompt, invoke, predict, predict_messages

In [ ]:
create_vector_db()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')